In [5]:
import pandas as pd
import pingouin as pg
import scipy.stats
import seaborn as sns
import requests
from urllib.parse import urlencode

In [6]:
# Чтение данных из облака
def download_link(public_key):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    return download_url

In [ ]:
df = pd.read_csv(download_link('https://disk.yandex.ru/d/vPXuAKmr2PraCw'))

In [ ]:
df

In [ ]:
df.query("group == 'A'").events

Проверьте, являются ли дисперсии внутри групп примерно одинаковыми (гомогенными) с помощью теста Левена.

In [ ]:
scipy.stats.levene(df.query("group == 'A'").events, df.query("group == 'B'").events, df.query("group == 'C'").events)

Также убедитесь, что данные распределены нормально, применив тест Шапиро-Уилка. Перед применением критерия при проверке на нормальность обязательно сделайте семплирование (1000 наблюдений), так как на больших выборках могут получаться неверные результаты. Для воспроизводимости результата используйте параметр random_state=17 в методе sample.

Дополнительно: попробуйте посчитать тест Шапиро-Уилка без использования метода sample. Сравните  результат с normaltest, а также qq-графиками. Какой вывод можно сделать по совокупности этих методов?

In [ ]:
pg.normality(df.query("group == 'A'").sample(1000, random_state=17), dv='events', group='group')\
    .sort_index()

In [ ]:
pg.normality(df.query("group == 'B'").sample(1000, random_state=17), dv='events', group='group')\
    .sort_index()

In [ ]:
pg.normality(df.query("group == 'C'").sample(1000, random_state=17), dv='events', group='group')\
    .sort_index()

попробуйте посчитать тест Шапиро-Уилка без использования метода sample. Сравните  результат с normaltest, а также qq-графиками. Какой вывод можно сделать по совокупности этих методов?

In [ ]:
pg.normality(data=df, dv='events', group='group')

In [ ]:
pg.qqplot(df.query("group == 'A'").events)
pg.qqplot(df.query("group == 'B'").events)
pg.qqplot(df.query("group == 'C'").events)

In [ ]:
pg.anova(df, dv='events', between='group')

Теперь используйте критерий Тьюки и определите, между какими группами есть статистически значимые различия. 

Дополнительно: попробуйте другие варианты поправок на множественные сравнения через pingouin. Дают ли они те же выводы, что и Тьюки?

In [ ]:
pg.pairwise_tukey(data=df, dv='events', between='group')

In [ ]:
pg.pairwise_gameshowell(data=df, dv='events', between='group')

Во втором эксперименте требуется проверить, как пользователи отреагируют на изменение формата кнопки оформления заказа, с разбивкой по сегменту клиента.

In [ ]:
df_task_2 = pd.read_csv(download_link('https://disk.yandex.ru/d/-EtxKVetq2BRjw'))

Какой метод скорее всего пригодится в данном случае?

In [ ]:
df_task_2

Посмотрим на данные. Для начала визуализируйте распределение events для контрольной и тестовой групп. 

In [ ]:
sns.displot(data=df_task_2.query("group == 'test'"), x='events')
sns.displot(data=df_task_2.query("group == 'control'"), x='events')

Посчитайте среднее, медиану и стандартное отклонение events в разбивке по группе и по сегменту пользователя. 

В качестве ответа укажите стандартное отклонение для пользователей из сегмента high и контрольной группы, с точностью до 2 знаков после точки.

In [ ]:
df_task_2.groupby(['group', 'segment'], as_index=False)\
    .std()

Проведите тест (через anova_lm либо pg.anova), а затем используйте критерий Тьюки, чтобы понять между какими группами есть статистически значимые различия.

In [ ]:
pg.anova(data=df_task_2, dv='events', between=['group', 'segment'])

In [ ]:
df_task_2['combination'] = df_task_2.group.astype('str') + '/' + df_task_2.segment.astype('str')

In [ ]:
df_task_2.head()

In [ ]:
pg.pairwise_tukey(data=df_task_2, dv='events', between='combination')

In [ ]:
sns.pointplot(data=df_task_2, x='group', y='events')

In [ ]:
sns.pointplot(data=df_task_2, x='segment', y='events')